# Testing for qnmfit and related classes

In [9]:
# Low-level import 
from numpy import array,loadtxt,linspace,zeros,exp,ones,unwrap,angle,pi
# Setup ipython environment
%load_ext autoreload
%autoreload 2
%matplotlib inline
# Import useful things from kerr
from kerr import qnmfit,rgb
# Import usefult things from nrutils
from nrutils import scsearch,gwylm,scbuild,jf14067295,Mf14067295
# Setup plotting backend
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 0.8
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.labelsize'] = 20
from matplotlib.pyplot import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Find a Simulation 

In [177]:
# A = scsearch(keyword='hrq',verbose=True,unique=True,validate_remnant=True,q=1.2)
A = scsearch(keyword=('q18a0aM04c05v4_et05_2_T_80'),nonprecessing=True,verbose=True,unique=True,validate_remnant=True)
# A = scsearch(keyword='D11_q2.50_a-0.2_m200',verbose=True,unique=True,validate_remnant=True)
# for e in A:
#     e.mf,e.xf = Mf14067295(e.m1,e.m2,e.S1[-1]/(e.m1**2),e.S2[-1]/(e.m2**2)),jf14067295(e.m1,e.m2,e.S1[-1]/(e.m1**2),e.S2[-1]/(e.m2**2))

[scsearch]>> Found keyword (='q18a0aM04c05v4_et05_2_T_80') keyword.
[scsearch]>> Found nonprecessing (=True) keyword.
[scsearch]>> Found unique (=True) keyword.
[scsearch]>> Found validate_remnant (=True) keyword.
[scsearch]>> Found verbose (=True) keyword.
(scsearch)>> List of keywords or string keyword found: ALL scentry objects matching will be passed. To pass ANY entries matching the keywords, input the keywords using an iterable of not of type list.
## Found 1 unique simulations:
[0001][silures] BAM: 1chi0.40-saa-q18.00	(q18a0aM04c05v4_et05_2_T_80)



In [178]:
e = A[0]
print e.simdir()
print e.simname
print e.mf,e.xf
print Mf14067295(e.m1,e.m2,e.S1[-1]/(e.m1**2),e.S2[-1]/(e.m2**2)),jf14067295(e.m1,e.m2,e.S1[-1]/(e.m1**2),e.S2[-1]/(e.m2**2))
e.mf,e.xf = Mf14067295(e.m1,e.m2,e.S1[-1]/(e.m1**2),e.S2[-1]/(e.m2**2)),jf14067295(e.m1,e.m2,e.S1[-1]/(e.m1**2),e.S2[-1]/(e.m2**2))

/Volumes/athena/silures/BAM/q18/q18_a0_aM04/q18a0aM04c05v4_et05_2_T_80/
q18a0aM04c05v4_et05_2_T_80
0.899757623874 0.224200494264
0.994498750894 0.502071803042


In [181]:
a = A[0]
ll,mm = 2,1
Y = gwylm( a, lm=[ll,mm], clean=True, dt=0.5, verbose=True )
print a.mf,a.xf
a.mf,a.xf = Mf14067295(e.m1,e.m2,e.S1[-1]/(e.m1**2),e.S2[-1]/(e.m2**2)),jf14067295(e.m1,e.m2,e.S1[-1]/(e.m1**2),e.S2[-1]/(e.m2**2))
# Y.brute_masspin(apply_result=True)
# Y = gwylm( a, lm=[(ll,mm)], clean=True, dt=0.35 )

(gwylm)>> Found clean (=True) keyword.
(gwylm)>> Found dt (=0.5) keyword.
(gwylm)>> Found lm (=[2, 1]) keyword.
(gwylm)>> Found scentry_obj (=<nrutils.core.nrsc.scentry instance at 0x13c23a5f0>) keyword.
(gwylm)>> Found verbose (=True) keyword.
(gwylm)>> The (extraction_parameter,level) is (2,5), which differs from the config values of (5,5). You have either manually input the non-config values, or the handler has set them by looking at the contents of the simulation directory. 
(gwylm!)>> The l=m=2 multipole will be loaded in order to determine important characteristice of all modes such as noise floor and junk radiation location.
(__make_lmlist__)>> The following spherical multipoles will be loaded:[(2, 1), (2, 2)]
(load)>> Loading: psi3col.r2.l5.l2.m1.gz
(3,)
[ 0.  0.  0.]


ValueError: 'axis' entry is out of bounds

In [ ]:
ax,fig = Y.lm[ll,mm]['strain'].plot(domain='freq')
# ax[0].set_xlim([2620,2780])
# ax[0].set_yscale('log')
# ax[0].set_ylim([1e-4,0.1])
# from kerr import leaver
for a in ax:
    a.axvline( leaver(Y.xf,2,2,Mf=Y.mf)[0].real/(2*pi),color='r',linestyle='--', label='(2,2)' )
    a.axvline( leaver(Y.xf,3,3,Mf=Y.mf)[0].real/(2*pi),color='b',linestyle='--', label='(3,3)' )
    a.axvline( leaver(Y.xf,4,4,Mf=Y.mf)[0].real/(2*pi),color='g',linestyle='--', label='(4,4)' )
    a.axvline( leaver(Y.xf,5,5,Mf=Y.mf)[0].real/(2*pi),color='m',linestyle='--', label='(5,5)' )
ax[0].legend(frameon=False,loc='upper left')

In [ ]:
Y.calcflm()

hh = Y.lm[2,2]['strain']
# print h.intrp_t_amp_max

ff = Y.lm[2,2]['news']
print hh.intrp_t_amp_max-ff.intrp_t_amp_max


## Try to fit the ringdown using the qnmfit class

In [ ]:
close('all')
dtfh = hh.intrp_t_amp_max-ff.intrp_t_amp_max
y = Y.ringdown(T0=20,T1=None,use_peak_strain=not True,verbose=True)

#
j=0
# print Y.xf,Y.mf
g = y.lm[ll,mm]['psi4']

f = qnmfit( g, verbose=True, greedy=True, prange=None )
#
f.plot(imrgwfo=Y.ylm[j])
# f.plot_beta_arr()

# print f.homez
print abs(f.iamap[f.homez])
print f.xf
print f.homez

f.plot_beta_arr()